In [163]:
import os
import json
#import re
import shutil
#import pandas
from collections import Counter

In [243]:
data_file = "Downloads/[name of discord server - chat channel].json"
data_folder = data_file+"_Files"

#option 1 - collect all the attachments based on the minimum number of total emojis the post has
minimum_emoji_reacts = 15

#option 2 - collect all the attachments based on the minimum number of time the same filename was posted
minimum_num_reposts = 15

In [223]:
#functions
# need a couple things from each message
# does it has a file?
# how many total reactions
# determ
def get_attachments(message):
    if "attachments" in message.keys():
        return message['attachments']
    else:
        return []

def get_filename(attachment):
    #return attachment['fileName']
    #return attachment['url']
    filename = attachment['url']
    filename = filename[filename.rfind("\\")+1:]
    return filename

def get_truncated_filename(filename):
    truncated = filename[:filename.rfind("-")]+filename[filename.rfind("."):]
    if (len(truncated)-len(filename))!=6:  # reduction should be exactly 6. if it's not, then it's the wrong string
        #print(f"issue truncating filename: {filename}")
        return filename
    else:
        return truncated

def de_truncate_filename(truncated, all_files):
    #return truncated[:truncated.rfind(".")]
    return [i for i in all_files if truncated[:truncated.rfind(".")] in i][0]

def get_total_reactions(message):
    total_reacts = 0
    for reactions in message['reactions']:
        total_reacts+=reactions['count']
        
    return total_reacts

print(data['messages'][0]['reactions'][0]['count'])



17


In [ ]:
#load the json
data = json.load(open(data_file,encoding="UTF-8"))

In [244]:
#Option 1, get list of files above x reactions
minimum_emoji_reacts
keep_files_set = set()
keep_files = []
total_files = 0

for message in data['messages']:
    files = get_attachments(message)
    total_files+=len(files)
    reacts = get_total_reactions(message)
    if reacts < minimum_emoji_reacts:
        continue
    for file in files:  # we want to keep this file
        filename = get_filename(file)
        truncated = get_truncated_filename(filename)
        if truncated not in keep_files_set:  # unless we already kept a duplicate
            keep_files.append(filename)
            keep_files_set.add(truncated)

print(f"{len(data['messages'])} total messages")
print(f"{total_files} total files")
#print(len(keep_files_set))
print(f"{len(keep_files)} files to keep (minimum {minimum_emoji_reacts} emoji reactions)")

print(100-len(keep_files)/
     (len(data['messages'])+len(keep_files))*100)

#make folder for the copied files
dest_dir = f"min_reacts_{minimum_emoji_reacts}" 
try:
    os.mkdir(dest_dir)
except:
    shutil.rmtree(dest_dir)
    #os.rmdir(dest_dir)
    os.mkdir(dest_dir)
for file in keep_files:
    try:
        src = os.path.join(data_folder,file)
        #dst = os.path.join(dest_dir, truncate_filename(file))
        dst = os.path.join(dest_dir, file)
        shutil.copy(src,dst)
    except:
        print(f"couldn't find file {file}")
    
print("done")
print(f"{len(os.listdir(dest_dir))} files copied")

229497 total messages
37258 total files
429 files to keep (minimum 15 emoji reactions)
99.8134182302132
done
429 files copied


In [242]:
#Option 2, copy files that are reposted minimum_num_reposts times
c = Counter()
total_files = 0
all_files = os.listdir(data_folder)

#all_files = os.listdir(data_folder)
#all_files = [truncate_filename(i) for i in all_files]

for message in data['messages']:
    files = get_attachments(message)
    files = [truncate_filename(get_filename(i)) for i in files]
    total_files += len(files)
    c.update(files)

    
#keep posts reposted x times
keep_files = []
for i in c.keys():
    if c[i]>=minimum_num_reposts:
        keep_files.append(i)
        
print(f"{total_files} total files")
#print(len(keep_files_set))
print(f"{len(keep_files)} files to keep (minimum {minimum_num_reposts} reposts)")

print(100-len(keep_files)/
     (len(data['messages'])+len(keep_files))*100)

#make folder for the copied files
dest_dir = f"min_reposts_{minimum_num_reposts}" 
try:
    os.mkdir(dest_dir)
except:
    shutil.rmtree(dest_dir)
    #os.rmdir(dest_dir)
    os.mkdir(dest_dir)
for file in keep_files:
    try:
        new_file = de_truncate_filename(file,all_files)
        src = os.path.join(data_folder,new_file)
        #dst = os.path.join(dest_dir, truncate_filename(file))
        dst = os.path.join(dest_dir, new_file)
        shutil.copy(src,dst)
    except:
        print(f"couldn't find file {src}")
    
print("done")
print(f"{len(os.listdir(dest_dir))} files copied")

37258 total files
225 files to keep (minimum 15 reposts)
99.902055527986
done
214 files copied


In [129]:
#check that all filenames exist
count_bad = 0
count_good = 0
good_set = set()
all_files = os.listdir(data_folder)
all_set = set(all_files)
for message in data['messages']:
    files = get_attachments(message)
    for file in files:  # we want to keep this file
        filename = get_filename(file)
        #filename=os.path.join(data_folder,filename)
        if filename not in all_files:
            print(f"{filename}")
            count_bad+=1
        else:
            count_good+=1
            good_set.add(filename)
                  
print(count_good)
print(count_bad)

37258
0


In [121]:
f = filename[filename.rfind("\\")+1:]
#os.path.isfile(os.path.join(data_folder,f))
os.path.isfile(filename)


False